# Trainingspipeline 2

- ohne Bigramme
- vector_size: 300
- window: 5
- min_count: 2
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [6]:
df = pd.read_csv('../../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [7]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
0,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv . 11 . rima del arcade : tirsi leucasio ( g...
1,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,"iv . 5 . Lorenzo magalotti , lettere odoroso (..."
2,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit..."
3,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...
4,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit..."


In [8]:
df.shape

(304129, 10)

In [9]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [10]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [11]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1985']
df10 = df_periods['1986-2000']
df11 = df_periods['2001-2021']

In [12]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
74265,diacorisBiagi1.txt,DiaCoris,Enzo Biagi,L'alluvione del Polesine,1951.0,1951-1985,stampa,L ' angoscia di un padre Si parla di battaglie...,152.0,l ' angoscia di uno padre si parlare di battag...
74266,LISCastiglia1.txt,LIS,Roderigo di Castiglia,«Vittorini se n'è ghiuto/ E soli ci ha lasciat...,1951.0,1951-1985,stampa,"A dire il vero , nelle nostre file pochi se ne...",1695.0,"a dire il vero , nella nostro file pochi se ne..."
74267,LISBianchi1.txt,LIS,Ranuccio Bianchi Bandinelli,Il nostro lavoro nella scuola,1951.0,1951-1985,stampa,"Come in tutti i congressi , anche nel VII Cong...",2898.0,"come in tutto il congresso , anche nel vii con..."
74268,LLAlbertelli1.txt,Liber Liber,Pilo Albertelli,Rousseau,1951.0,1951-1985,prosa letteraria,﻿Pilo Albertelli\nRousseau\n\n Nacque il 28 g...,4894.0,﻿pilo albertelli \n rousseau \n\n nascere il...
74269,LISJotti1.txt,LIS,Nilde Jotti,La questione dei fumetti,1951.0,1951-1985,stampa,Il dibattito sulla stampa a fumetti per i raga...,2785.0,il dibattito sulla stampa a fumetto per il rag...


## Training von Word2Vec

In [13]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [14]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 5                         # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

### Zeitraum 1: 1700-1750

In [16]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [17]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 20.9 s


In [18]:
print(sentences1[3])

['lungo', 'inospita', 'campagna', 'aggiungere', 'invidia', 'lato', 'manco', 'dire', 'anch', 'essere', 'teco', 'labbro', 'bianco', 'veggo', 'veneno', 'cor', 'stagno']


In [19]:
len(sentences1)

113328

In [20]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 55s


In [21]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7626875042915344),
 ('desolazione', 0.7543124556541443),
 ('rumore', 0.748366117477417),
 ('macello', 0.7454737424850464),
 ('riempiè', 0.7368387579917908),
 ('sacco', 0.7356059551239014),
 ('sconfiggere', 0.7353587746620178),
 ('milanesi', 0.7263142466545105),
 ('costernazione', 0.7245988845825195),
 ('eccidio', 0.7211870551109314)]

In [22]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2Vec200', '20w2v1.model'))

### Zeitraum 2: 1751-1800

In [23]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [27]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 18.3 s


In [28]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 43s


In [29]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('atterrire', 0.8720303177833557),
 ('ruina', 0.869418203830719),
 ('usurpatore', 0.8663512468338013),
 ('respingere', 0.8654507398605347),
 ('letargo', 0.8637111186981201),
 ('colmo', 0.860849916934967),
 ('cadde', 0.8604916930198669),
 ('avviliti', 0.8581162691116333),
 ('tumulto', 0.8524579405784607),
 ('spontaneo', 0.847689151763916)]

In [31]:
w2v2.save(os.path.join('trained_models/Word2Vec200', '20w2v2.model'))

### Zeitraum 3: 1801-1825

In [32]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [33]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 17.4 s


In [34]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 56s


In [35]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('avversa', 0.8423526883125305),
 ('furore', 0.8406038880348206),
 ('oppressore', 0.8366074562072754),
 ('infiammare', 0.8363872766494751),
 ('avvampare', 0.8336716294288635),
 ('rissa', 0.832763135433197),
 ('orrore', 0.8302266001701355),
 ('turpe', 0.8301956057548523),
 ('languente', 0.82794588804245),
 ('crudeltà', 0.8278446197509766)]

In [36]:
w2v3.save(os.path.join('trained_models/Word2Vec200', '20w2v3.model'))

### Zeitraum 4: 1826-1850

In [37]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [38]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 16.6 s


In [39]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 45s


In [40]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('furore', 0.8164554238319397),
 ('opprimere', 0.8009040355682373),
 ('spavento', 0.7982795834541321),
 ('indegnazione', 0.7791193723678589),
 ('smisurato', 0.7775142788887024),
 ('orgoglio', 0.7741279602050781),
 ('imminente', 0.7720504403114319),
 ('gelosia', 0.7697895169258118),
 ('odio', 0.768740177154541),
 ('impeto', 0.7639153003692627)]

In [41]:
w2v4.save(os.path.join('trained_models/Word2Vec200', '20w2v4.model'))

### Zeitraum 5: 1851-1875

In [42]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [43]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 18.8 s


In [44]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 36s


In [45]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('fremito', 0.8322952389717102),
 ('sopraffare', 0.8256614208221436),
 ('orrore', 0.8221233487129211),
 ('raccapriccio', 0.8211771845817566),
 ('spavento', 0.8211342096328735),
 ('disinganno', 0.8141425848007202),
 ('represso', 0.8139486312866211),
 ('brivido', 0.8138133883476257),
 ('sgomento', 0.812529444694519),
 ('tedio', 0.8121728897094727)]

In [46]:
w2v5.save(os.path.join('trained_models/Word2Vec200', '20w2v5.model'))

### Zeitraum 6: 1876-1900

In [47]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [48]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 18.6 s


In [49]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 32s


In [50]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.796788215637207),
 ('angoscia', 0.775015652179718),
 ('sgomento', 0.7698339223861694),
 ('tremendo', 0.7667927742004395),
 ('orrore', 0.760362446308136),
 ('disperazione', 0.7600436210632324),
 ('ira', 0.7599384784698486),
 ('crisi', 0.7590783834457397),
 ('furore', 0.7574854493141174),
 ('incutere', 0.7498072385787964)]

In [51]:
w2v6.save(os.path.join('trained_models/Word2Vec200', '20w2v6.model'))

### Zeitraum 7: 1901-1925

In [52]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [53]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 18 s


In [54]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 31s


In [55]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7944040894508362),
 ('indicibile', 0.7923570871353149),
 ('subitaneo', 0.783981204032898),
 ('indignazione', 0.7829123735427856),
 ('orrore', 0.7810711860656738),
 ('straziante', 0.7795203328132629),
 ('invocazione', 0.770779550075531),
 ('cordoglio', 0.7704339027404785),
 ('commozione', 0.7690957188606262),
 ('tumulto', 0.7676676511764526)]

In [56]:
w2v7.save(os.path.join('trained_models/Word2Vec200', '20w2v7.model'))

### Zeitraum 8: 1926-1950

In [57]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [58]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 17.1 s


In [59]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 37s


In [60]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.8519399762153625),
 ('ribrezzo', 0.8458722829818726),
 ('subitaneo', 0.8328276872634888),
 ('orrore', 0.817833662033081),
 ('indicibile', 0.8148026466369629),
 ('disperazione', 0.8113715052604675),
 ('raccapriccio', 0.8107553124427795),
 ('disgusto', 0.8060970306396484),
 ('stupore', 0.8057337999343872),
 ('furore', 0.8042652606964111)]

In [61]:
w2v8.save(os.path.join('trained_models/Word2Vec200', '20w2v8.model'))

### Zeitraum 9: 1951-1985

In [62]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [63]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 17.9 s


In [64]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 42s


In [65]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.7547872066497803),
 ('sconvolgere', 0.7504227757453918),
 ('orribile', 0.7362454533576965),
 ('immane', 0.7305439710617065),
 ('annientare', 0.724656343460083),
 ('seminare', 0.7246383428573608),
 ('sanguinoso', 0.7156408429145813),
 ('valanga', 0.7152332663536072),
 ('apprensione', 0.7146035432815552),
 ('sanguinario', 0.7143808007240295)]

In [66]:
w2v9.save(os.path.join('trained_models/Word2Vec200', '20w2v9.model'))

### Zeitraum 10: 1986-2000

In [67]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [68]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 36.8 s


In [69]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 36s


In [70]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('seminare', 0.8671962022781372),
 ('insanguinare', 0.8362770676612854),
 ('lutto', 0.8298245668411255),
 ('incubo', 0.8213831782341003),
 ('preda', 0.8209490776062012),
 ('sanguinoso', 0.8144978284835815),
 ('cieco', 0.8124772310256958),
 ('rabbia', 0.804171085357666),
 ('disperazione', 0.7971651554107666),
 ('inerme', 0.7934804558753967)]

In [71]:
w2v10.save(os.path.join('trained_models/Word2Vec200', '20w2v10.model'))

### Zeitraum 11: 2001-2021

In [72]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [73]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 12.6 s


In [74]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 52s


In [75]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('seminare', 0.8841827511787415),
 ('distruzione', 0.8683592677116394),
 ('incutere', 0.8559337258338928),
 ('soffocare', 0.8551183938980103),
 ('sgomento', 0.8538791537284851),
 ('orrore', 0.8518758416175842),
 ('disprezzo', 0.8489519357681274),
 ('demonio', 0.8467963933944702),
 ('disperazione', 0.8466752171516418),
 ('dèi', 0.8455513119697571)]

In [76]:
w2v11.save(os.path.join('trained_models/Word2Vec200', '20w2v11.model'))